In [36]:
#Import Libraries
from geopy.geocoders import Nominatim
import pandas as pd
import folium
import geopandas as gpd
import os
import tkinter as tk
from tkinter import Tk
from tkinter import filedialog

In [37]:
# Choose the input file using tkinter
Tk().withdraw()
file_path = filedialog.askopenfilename()

# Read in the Excel file as a pandas dataframe
df = pd.read_excel(file_path)

In [38]:
# Create geolocator object
geolocator = Nominatim(user_agent='my-app')

# Define the possible names of the address columns
address_columns = {
    'street_and_house_number': ['Addr1','Risk Address','Street'],
    'city': ['City','Risk City','city'],
    'state': ['State','StateProvCd','Risk State'],
    'postal_code': ['Risk Zip Code','PostalCd','Zip','Zip Code'],
    'latitude': ['Lat', 'Risk Lat', 'lat'],
    'longitude': ['Long','Risk Long','long']}

# Loop over the keys of the address_columns dictionary
for col_name in address_columns:
    # Check if any of the possible column names are present in the DataFrame
    possible_col_names = address_columns[col_name]
    matching_cols = df.columns[df.columns.isin(possible_col_names)]
    if len(matching_cols) > 0:
        # If a matching column is found, use the first one
        df[col_name] = df[matching_cols[0]]

# Loop over the keys of the address_columns dictionary
for col_name in address_columns:
    # Check if any of the possible column names are present in the DataFrame
    possible_col_names = address_columns[col_name]
    matching_cols = df.columns[df.columns.isin(possible_col_names)]
    if len(matching_cols) > 0:
        # If a matching column is found, use the first one
        df[col_name] = df[matching_cols[0]]

In [39]:
# Define function to get latitude and longitude from city and state
def get_lat_long(city, state):
    location = geolocator.geocode(f"{city}, {state}")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# Apply the function to the city and state columns to get the latitudes and longitudes
df['latitude'], df['longitude'] = zip(*df.apply(lambda x: get_lat_long(x['city'], x['state']), axis=1))

In [40]:
# Drop rows with missing latitude and longitude values
num_rows_before_drop = df.shape[0]
df.dropna(subset=['latitude', 'longitude'], inplace=True)
num_rows_after_drop = df.shape[0]

# Print percentage and count of rows dropped
num_rows_dropped = num_rows_before_drop - num_rows_after_drop
pct_rows_dropped = num_rows_dropped / num_rows_before_drop * 100
print(f"Dropped {num_rows_dropped} rows ({pct_rows_dropped:.2f}%) due to missing latitude and/or longitude values.")

Dropped 3 rows (0.30%) due to missing latitude and/or longitude values.


In [41]:
#Plot latitudes and longitudes on a map using folium

# Center the map on the United States and increase the zoom level
map_us = folium.Map(location=[39.50, -98.35], zoom_start=6)

# Add markers for each row in the Excel file and increase the marker size
for _, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(map_us)
map_us

In [47]:
state_counts = df['state'].value_counts(normalize=True) * 100

# Print the percentage of rows in each state
for state, count in state_counts.items():
    print(f"{state}: {count:.2f}%")

WI: 95.79%
MN: 1.40%
UT: 0.80%
AZ: 0.80%
IL: 0.60%
IA: 0.30%
MI: 0.10%
ID: 0.10%
NV: 0.10%


In [48]:
count_by_state = df['state'].value_counts()

# Print the count of rows in each state
print(count_by_state)

WI    955
MN     14
UT      8
AZ      8
IL      6
IA      3
MI      1
ID      1
NV      1
Name: state, dtype: int64
